In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 22 17:12:01 2021

@author: User
"""
# import 爬蟲所需要的東西
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import numpy as np
import os,sys,time
from PIL import Image
from urllib import request
import requests
import json
import sys
import modin.pandas as pd
from iteration_utilities import duplicates
import asyncio
import unittest
from numba import jit
# 淺複製（shallow copy）讓物件獨立
# 這樣這邊我會保留All_Graph_Name_List
# 跟All_Children_list
# 可以備考
# 之後再看看可不可以用非同步優化流程，釋放記憶體空間。
import copy
import csv
import networkx as nx
# P.S.
# python 預設調用  print， 會調用 __str__ 函數。
# __rerp__ 是預覽



# 實作　Node 單位


class Node:
    #建構子，初始化節點
    #初始authority跟 hub,值皆為1
    def __init__(self, ID): #一定會有self這個值，並用self代稱上面的class_name(Node)
        self.ID = ID
        self.parents = []
        self.children = []  #初始的children 跟 parents串列裡面都是空的。(因為都沒有建立任何連結)
        self.auth = 1.0
        self.hub = 1.0
    # __str__ 是print會調用的函數
    def __str__(self):
        return str(self.ID)
        
    def show(self): #所有的function裡面都要有self，代指使用自己方法的本身。
        print(f"This's {self.ID}")
        
    #link_child 跟 link_parent，為網站間的有向連結，但其並沒有考慮到各child節點或各parent節點之間加入到"children"或"parent"串列的先後順序。
    #只能計算此節點所有連出的節點個數計算。
        
    def link_child(self,new_child): #這Method中有兩個參數，self如上述，"new child" 為其他的Node類物件，近來這個程式的代稱為"new_child"
        for child in self.children: #在呼叫Method link_child的 Node 中，依序搜尋children 串列。
            if(new_child.ID == child.ID): #若已經有了，回傳None
                print("duplicate") #若有重複印出"duplicate"
                #print(child.ID)
                return None
        self.children.append(new_child)
        
    def link_parent(self,new_parent): #同上
        for parent in self.parents:
            if (new_parent.ID == self.ID):
                print("duplicate")
                #print(self.ID)
                return None
        self.parents.append(new_parent)
        
    def show_children(self): #印出一個節點的所有孩子(印出這個node所有連出的連結)
        if len(self.children) == 0:
            print("this node :",self.ID,", children_list is empty.")
        else:
            for i in range(len(self.children)):
                print(self.children[i].ID)
        
    def show_parents(self): #印出一個節點的所有parent(印出這個node所有連入的連結)
        for i in range(len(self.parents)):
            print(self.parents[i].ID)
            
    def update_hub(self):  #更新此點的hub分數
        update_score = 0.0
        for node in self.children:
            update_score += node.auth
        self.hub = update_score
        #print(self.hub)
        
    def update_auth(self):   #更新此點的authority分數
        update_score = 0.0
        for node in self.parents:
            update_score += node.hub
        self.hub = update_score
        #print(self.auth)
    
    def show_indegree(self):  #算出所有此點的 in_degree
        pass
    
    def show_outdegree(self):  #算出所有此點的 out_degree
        pass
    
    def normalize_hub_score(self):   #將此點的hub分數除以全部的hub分數總合
        pass
    
    def normalize_authority_socre(self):   #將此點的authority分數除已全部的authority分數總合。
        pass
    
#實作 Graph

class Graph:   #在這個網絡上(圖)的所有網站(點)集合
    def __init__(self, G_name):
        self.name = G_name  #圖的名字
        self.nodes = [] #一開始的nodes集合為空
        self.edges = [] #所有的邊集合我也設一個串列存起來，但這邊我目前還沒有想到怎麼加入可以加入有向邊。
    
    def insert_node(self, node):  #將外來的點加入這個圖的點集合中
        try:    
            self.nodes.append(node)
            # print("successfully insert！")
        except error as e:
            print(f"insert fail:{e}")
    
    def display_hub_authority_score(self):
        pass
    
    def find_node(self, ID):  # 要找的點的名字，也就是網址
        for node in self.nodes: # 在自己圖中的節點做尋找
            if node.ID == ID:
                # print("found")
                return node # 找到的話就回傳節點
        else: # 否則就說沒找到
            # print("Not found")
            return None # 回傳空值
    def normalize_auth_hub(self):
        authority_sum = 0.0
        hub_sum = 0.0
        for node in self.nodes:
            authority_sum += node.auth
        for node in self.nodes:
            hub_sum += node.hub

        for node in self.nodes:
            node.auth /= authority_sum
            node.hub /= hub_sum
            
    def get_auth_hub_list(self):
        auth_list = np.asarray([node.auth for node in self.nodes], dtype='float32')
        hub_list = np.asarray([node.hub for node in self.nodes], dtype='float32')

        return np.round(auth_list, 3), np.round(hub_list, 3)

    def get_pagerank_list(self):
        pagerank_list = np.asarray([node.pagerank for node in self.nodes], dtype='float32')
        return np.round(pagerank_list, 3)


# 將所有 ID 一樣的都刪除。
def set_node_list(list_a):
    # 比較所有的ID名稱，先用for迴圈將所有的ID名稱存起來。
    all_name = [] #所有名字的列表
    for i in list_a:
        all_name.append(i.ID)
    all_name_count = [] #計算每個網站重複的數量
    for index,i in enumerate(list_a):  #對每個list_a 的元素做檢查
        count = 0
        for k in all_name: # 在所有的名字中搜尋，如果有一樣的名字，count就加一。
            if i.ID == k:
                count +=1
        all_name_count.append(count)
    for index,i in enumerate(all_name_count):  #超過一次以上的就刪除
        if i > 1:
            del list_a[index]
            del all_name[index]
            del all_name_count[index]
    return list_a                   #回傳一list 其為不重複的 node版本

def search_all_link(query_link):
    a = []  # 這個網站所有的 outlink
    html = requests.get(query_link, timeout = 10)
    # 為防止爬蟲卡死情況(request 不會自行斷開)，故上面設置 timeout 參數，設定固定時間若未完成則 斷開連結。
    html.encoding = 'UTF-8' #使用html編碼。
    sp = BeautifulSoup(html.text, 'html5lib') #指定'html5lib'作為解析器
    #若只使用sp.link 則只會找出第一筆
    #故這邊要使用"find.all"語法
    link_search = sp.find_all("a") #找出所有的 <a> (超連結)
    for i in range(len(link_search)):
        k = str(link_search[i].get("href"))  #這行一定要記得要轉type！！！轉成str不然有時候會無法切片取值。
        if k[0:5] == "https": #只留下"https"
            a.append(k)
    return a  #回傳一列網址(type list)

#先將所有node_list中的資料轉換成type str存入到list再寫入檔案
def node_to_text(all_node_list):
    node_list_txt = [] # node_list 的所有內容
    for i in all_node_list:
        tmp_list = [] # 初始化
        # 依序記錄結構化的node的每個資料
        tmp_list.append(i.ID)
        # 這邊嘗試寫遞迴函式，不停往下找
        # 加入cildren
        tmp_children = []
        for j in i.children:
            tmp_children.append(j.ID)
        tmp_list.append(tmp_children)
        # 加入parents
        tmp_parents = []
        for k in i.parents:
            tmp_parents.append(k.ID)
        tmp_list.append(tmp_parents)
        tmp_list.append(i.auth)
        tmp_list.append(i.hub)
        # 再加入list
        node_list_txt.append(tmp_list)
    return node_list_txt  # 回傳一個 list
def listDups(tmp_list_aaa): # 記錄重複的功能
    return list(duplicates(tmp_list_aaa))
def str_to_url(aaa): # aaa 為文字格式。
    bbb = aaa.split(",") # 利用逗號做切分 # bbb為list格式 # 會回傳一個list
    flag_head = False # 一開始起頭開關為False
    tmp_word_list = [] # 裝暫存要轉變成文字的網址，每次只會存一個網址，之後就會被清空
    tmp_url_list = [] # 裝整理好的網址
    for url in bbb: # 在這bbb串列中做遞迴
        for char in url:
            if char == "'": # 當遇見"單引號"時switch (此時Flag_head只有兩種狀態)
                flag_head = not(flag_head) # 轉換開關
                # tmp_word_list.append(char) # 將其收錄至暫存的字串列。
            if flag_head == True:
                tmp_word_list.append(char) # 因為只有h會將其打開，所以一路收錄char 至 tmp_word_list
            if flag_head == False:
                StrA = "".join(tmp_word_list)
                StrA = StrA[1:]  # 去掉前端的單引號。 
                if len(tmp_word_list) != 0: # 不為空才加入
                    tmp_url_list.append(StrA)
                tmp_word_list = [] # 清空tmp_word_list
    # 所以我只要再把這些網址NODE化之後再加入CHILDREN串列就好 =)
    return tmp_url_list #回傳整理好的網址

# 111/1/1發現這邊有問題，雖然建立的nodes名字都一樣，但是因為是從不同檔案load進來的，會有問題。
# 111/1/1目前解決方法，隨然load進來的是不同檔案，但是會load到同個圖中。
def load_node_from_csv_out_side(start_num, end_num, df_name): # 讀csv檔的資料 # 並且加入子節點 # 也同時加入父節點
    # 參數分別為，load加入的檔案的第幾筆到第幾筆，讀入黨案名稱。
# 每個點依序建立連結
    # 在這邊才讀取檔案是為了要避免讀取全域變數
    df = pd.read_csv(df_name)
    start_time = time.time()  #開始執行時間
    for i in range(start_num, end_num + 1): # 先存1-9000筆
        bbb = df["2"][i] # 將一整串str去頭去尾。
        url_list = str_to_url(bbb)  # 會回傳一個整理好的串列
        tmp2 = Graph_all.nodes[i] # 選取第幾個node
        for j in url_list: # 對找到的每個網站
            # 要先觀察 "j" 這個網站名字，是不是已經存在在原有的193721筆資料中。
            a = Graph_all.find_node(j)
            if a != None: # 有找到值，代表此網站已經在圖中了。此時 a 就已經是圖中的節點。 type == node
                tmp2.link_child(a)
                a.link_parent(tmp2)
            else: # 沒找到值，代表這是新的點
                tmp_node = Node(j) # 先將網站節點化
                Graph_all.insert_node(tmp_node) # 圖中也要加入這個新的點。
                Graph_all.nodes[i].link_child(tmp_node) # 加入node化的子節點。 # 這邊確認沒有重複。
                tmp_node.link_parent(tmp2) # 連結原節點。
    print("花費:" + str(float(time.time() - start_time)) + "秒")
    print("finish")
def Diff(li1, li2): 
    return (list(set(li1).symmetric_difference(set(li2))))  
# 判別兩列表不同元素
def diff(list_a, list_b):
    list_1 = list_a.copy()
    list_2 = list_b.copy()
    intersection = [x for x in list_1 for y in list_2 if x == y]
    return intersection

def load_out_link(start, end, csv_file_name): # 讀所有out_link # 回傳一列表
    
    df = pd.read_csv(csv_file_name)
    return_list = [] # 要回傳的列表
    for index in range(start,end+1): # 將前9000筆資料加入列表中
        bbb = df["2"][index]
        url_list = str_to_url(bbb)
        for j in url_list:
            if j[0:4] == "http": # 等於http的才加入。
                #print(j) # 撿查
                return_list.append(j) # 依序加入
    return return_list

# 實行二分搜尋
# 定義：在排序好的數列裡，找某 target，找到就回傳其 index，否則回傳 -1
def binary_search(data_list, key_data):  #要加入的圖的點的列表(已知且已排序), 要尋找的 資料
    #設置選取範圍的指標
    low = 0
    upper = len(data_list) - 1
    while low <= upper:
        mid = int((low + upper) / 2) #取中間索引的值
        if data_list[mid].ID < key_data:    #若搜尋值比中間的值大，將中間索引+1，取右半
            low = mid + 1
        elif data_list[mid].ID > key_data:  #若搜尋值比中間的值小，將中間索引+1，取左半
            upper = mid - 1
        else:                    #若搜尋值等於中間的值，則回傳
            return mid # 回傳的是索引值。
    return "Node Not Found!"


#  這邊一定要針對外部的圖進行修改，無法不用全域變數。
#  這邊其實已經解決先後順序的問題了，所以可以嘗試看看非同步的寫法。
#  此函式適用於所有load進來的檔案中已經沒有新node(所有的不重複網站名稱都已經存在在以建立之圖中)
#  之後應該要建立一另外之函式for 新加入的網站(以上述的csv檔案儲存)
def build_module_all_nodes_found(original_url, graph, out_link_list_http): 
    # 確定全部的點都加入了才可以使用此函式
    # 參數 原始的網站，要加入的圖(裡面的點已排序)(對這個圖進行操作)，out_link列表
        # 找到原始網站的那個點
        # 0.22s
        # 這邊改用 binary search
    All_graph_nodes_list = graph.nodes # 使用那個圖的點 #  指的是同一個位置 # 這個圖已經排序了
    key_1 = binary_search(All_graph_nodes_list, original_url)
    node_tmp = All_graph_nodes_list[key_1] 
   # print("網站",node_tmp.ID)
    for out_url in out_link_list_http:
        if len(out_link_list_http) != 0:
            key_2 = binary_search(All_graph_nodes_list, out_url)
            # 0.22s
            if key_2 != "Node Not Found!":
                # 利用指標操作找到那個點
                node_tmp_2 = All_graph_nodes_list[key_2] 
                node_tmp.link_child(node_tmp_2)
                node_tmp_2.link_parent(node_tmp)
             #   print("out_link", node_tmp_2.ID)
        else:
            print("empty_outlink")
            
# 注意！目前存入csv檔案中的圖，其節點排序會跟原本不一樣，是無序的，但不影響之後運算。
# 將目前所作的圖，暫存到一個csv檔案中，以便後續利用。
def load_Garph_to_csv(Graph):
    # 參數說明： Graph: 要加入的圖、從第幾個點開始、到第幾點結束。
    Graph_nodes = []
    count = 0
    for node in Graph.nodes:
        every_node = []
        every_node.append(count)
        every_node.append(node.ID)
        tmp_parent = []
        for url in node.parents:
            tmp_parent.append(url.ID)
        every_node.append(tmp_parent)     # list格式
        tmp_children = []
        for url in node.children:
            tmp_children.append(url.ID)
        every_node.append(tmp_children)  # list格式
        every_node.append(node.auth)  # int
        every_node.append(node.hub)   # int
        #上面加完每點的各元素。
        count += 1
        Graph_nodes.append(every_node) # 在將每個點加入圖點總列表中
        #print("Graph_node append successfully")
    # 至此為止應該會將四個list都完整加入完成
    with open('Graph.csv', 'w', newline='', encoding='UTF-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL, delimiter=';')
        writer.writerows(Graph_nodes)

def load_csv_to_Graph(csv_file_name):
    csv.field_size_limit(500 * 1024 * 1024)
    with open(csv_file_name, newline='', encoding = "utf-8") as csvfile:
        # 讀取 CSV 檔案內容
        rows = csv.reader(csvfile, delimiter=';') # 設定分隔號

        # 以迴圈檢查檢查前11筆
        count = 0
        for row in rows:
            if count == 11:
                break
            # print(row)
            # print(type(row)) # <class 'list'>
            print(row[1]) # <class 'str'> # original url
            print(row[2]) # <class 'str'> # parent list
            print(row[3]) # <class 'str'> # children list
            print(row[4]) # <class 'str'> # auth score
            print(row[5]) # <class 'str'> # hub score
            count += 1
        HITS = Graph("Authority & Hub")
        all_name = [] # 紀錄名字以便排序
        for row in rows:
            all_name.append(row[1])
        Sorted_all_name = sorted(all_name)
        for name in Sorted_all_name:
            tmp_node = Node(name)
            HITS.insert_node(tmp_node) # 此時確保插入皆是排序過後的資料
    return HITS

def HITS_one_iter(graph):
    for node in graph.nodes:
        node.update_auth()
    for node in graph.nodes:
        node.update_hub()
        
    graph.normalize_auth_hub()
    
    
def binary_search_for_nx(data_list, key_data):  #要加入的圖的點的列表(已知且已排序), 要尋找的 資料
    #設置選取範圍的指標
    low = 0
    upper = len(data_list) - 1
    while low <= upper:
        mid = int((low + upper) / 2) #取中間索引的值
        if data_list[mid] < key_data:    #若搜尋值比中間的值大，將中間索引+1，取右半
            low = mid + 1
        elif data_list[mid] > key_data:  #若搜尋值比中間的值小，將中間索引+1，取左半
            upper = mid - 1
        else:                    #若搜尋值等於中間的值，則回傳
            return mid # 回傳的是索引值。
    return "Node Not Found!"

def top_n_pagerank_for_nx(p, graph, n):
    L = sorted(p.items(), key = lambda item:item[1], reverse=True)
    # 所以上面這一行會做出一個list包著tuple如下(根據value值排列)。
    num = 0
    for i in range(n):
        print("Top", num+1, graph.nodes[L[i][0]])
        print("weight:", L[i][1])
        num += 1
def top_n_authority_for_nx(a, graph, n): 
    # a 為一個dictionary物件，有網站代碼跟其權重值 EX:(1294317, 0.0028239679462683694)
    # grpah 為 nx 套件的圖
    # n: 找前幾名 (int)
    L = sorted(a.items(), key = lambda item:item[1], reverse=True)
    # 所以上面這一行會做出一個list包著tuple如下(根據value值排列)。
    num = 0
    for i in range(n):
        print("Top", num+1, graph.nodes[L[i][0]])
        print("weight:", L[i][1])
        num += 1
def top_n_hub_for_nx(h, graph, n):
    L = sorted(h.items(), key = lambda item:item[1], reverse=True)
    # 所以上面這一行會做出一個list包著tuple如下(根據value值排列)。
    num = 0
    for i in range(n):
        print("Top", num+1, graph.nodes[L[i][0]])
        print("weight:", L[i][1])
        num += 1

In [2]:
HITS = nx.DiGraph() # 建立一個叫HITS的有向圖

In [3]:
HITS.add_node(1, name = "n1", auth = 1.0, hub = 1.0)

In [4]:
HITS.add_node(2, name = "n1") 

In [5]:
print(HITS.nodes()) # 檢視點，注意，同id並不會被重複加入！ 同名的只會有唯一一個點

[1, 2]


In [6]:
print(HITS._node) # 檢視點屬性 點的屬性用{大括號包起來}

{1: {'name': 'n1', 'auth': 1.0, 'hub': 1.0}, 2: {'name': 'n1'}}


In [7]:
# 建立所有點的圖 
csv.field_size_limit(500 * 1024 * 1024)
with open("Graph0205.csv", newline='', encoding = "utf-8") as csvfile:
    # 讀取 CSV 檔案內容
    rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
    # 以迴圈檢查檢查前11筆
    #count = 0,
    for row in rows:
        #if count == 10000:  \n",
        #    break\n",
        # print(row)\n",
        # print(type(row)) # <class 'list'>\n",
        #print(row[1]) # <class 'str'> # original url\n",
        #print(row[2]) # <class 'str'> # parent list\n",
        #print(row[3]) # <class 'str'> # children list\n",
        #print(row[4]) # <class 'str'> # auth score\n",
        #print(row[5]) # <class 'str'> # hub score\n",
        HITS.add_node(int(row[0]) ,id = int(row[0]) ,original_url = row[1])
        #count += 1,

In [8]:
HITS.nodes[4]

{'id': 4,
 'original_url': 'http://www.google.com/books%3Fid%3DSMdiDwAAQBAJ%22%7D&client=17&contact_type=anno&source=partner-google-print'}

#  這邊一樣要有所有的網站列表

In [9]:
total_url = []
path = "url_list.txt"
f = open(path, "r",encoding ="utf-8")
level_0 = f.readlines()
for i in level_0:
    total_url.append(i[:-1])

total_url = list(set(total_url))
print(len(total_url))
counter=0

353


In [10]:
csv.field_size_limit(500 * 1024 * 1024)
for i in range(9): # 有幾個檔案range裡面就放幾
    this_level_total_url_list = []
    counter = i
    file_name = 'Asynchronous_crawler_level_'+ str((counter+1)) +'.csv' 
    with open(file_name, newline='', encoding = "utf-8") as csvfile:
        # 讀取 CSV 檔案內容
        rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
        for row in rows:
            if len(row) == 0: # 空行跳過
                continue
            if row[0][0:4] == "http":
                this_level_total_url_list.append(row[0]) # 原始url也要加進去
            children_list = str_to_url(row[2]) # 每個url的out_link
            for url in children_list:
                if url[0:4] == "http":
                    this_level_total_url_list.append(url)
        total_url += this_level_total_url_list
        total_url = list(set(total_url))
        print(len(total_url)) # 已經是不重複的了

11744
83821
445237
679822
849136
1110631
1350738
1530606
1754370


In [16]:
# 排序 以便進行binary search 演算法
Sorted_All_list = sorted(total_url)
#總長度
print(len(Sorted_All_list))
# 一樣檢查前10筆
Sorted_All_list[:10]

1754370


['http://news.ucsc.edu/2022/01/covid-reminders.html?ref=share',
 'http://news.ucsc.edu/2022/01/covid-reminders.html?ref=share&via=ucsc',
 'http://webdam.inria.fr/Jorge/files/wdm-yago.pdf" ',
 'http://www.facebook.com',
 'http://www.google.com/books%3Fid%3DSMdiDwAAQBAJ%22%7D&client=17&contact_type=anno&source=partner-google-print',
 'http://www.google.com/books%3Fid%3DsLJXV_z8XC4C%22%7D&client=17&contact_type=anno&source=partner-google-print',
 'http://www.nap.edu/download.php?record_id=13564',
 'http://www.thecanadianencyclopedia.ca/en/article/physics',
 'https%3A%2F%2Fbottlebarn.com%2Fproducts%2Ffruitlab-ginger-750ml',
 'https%3A%2F%2Fbottlebarn.com%2Fproducts%2Ffruitlab-grand-poppy-amaro-liqueur-750ml']

In [12]:
start_time = time.time()
# 開始建立關係
csv.field_size_limit(500 * 1024 * 1024)
with open("Graph0205.csv", newline='', encoding = "utf-8") as csvfile: # 開啟之前做的圖檔
    # 讀取 CSV 檔案內容
    count = 0
    rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
    for row in rows:
        #if count == 100000:
        #    break
        children_list = str_to_url(row[3])
        if len(children_list) != 0: #out_link集合不為0才做
            for out_link in children_list:
                index = binary_search_for_nx(Sorted_All_list, out_link)
                # print(binary_search_for_nx(all_original_list, row[1]))
                if index != "Node Not Found!":
                    #print(row[0]) #row[0] <class: str>
                    original_id = int(row[0])
                    HITS.add_edge(original_id, index)  # type int, type int
                    # 上面一行加入的點為 原始的id 跟 index
                    # 注意，這一行如果新增的點不再圖中，會自動新增   
        #count += 1
print("花費:" + str(float(time.time() - start_time)) + "秒")

花費:85.81551766395569秒


In [13]:
#HITS.add_edge(205,206)

In [14]:
# 隨機檢查依現存的點。
binary_search_for_nx(Sorted_All_list, 'https://xn--9i1b14l89cjqd521a.money-sang.co.kr/%EA%B3%A8%EB%93%9C%EC%9C%99%EB%A8%B8%EB%8B%88%EC%83%81/')

1742646

# HITS

In [15]:
start_time = time.time()
h,a=nx.hits(HITS) # 得到兩個dictionary(未排序)
print("花費:" + str(float(time.time() - start_time)) + "秒")

花費:16.714186191558838秒


In [17]:
# 分別印出前top 10 Authority & Hub
L = sorted(a.items(), key = lambda item:item[1], reverse=True) # reverse=True 降冪排列 大->小 ,  lambda 參數表:運算式, key = 主要用來比較的元素
# 所以上面這一行會做出一個list包著tuple如下(根據value值排列)。
print(L[:10])
print(type(L[:10][1]))

[(1294317, 0.0028239679462683694), (612776, 0.00282080268853487), (165648, 0.0028165672876734734), (612777, 0.0028150531192043036), (654094, 0.0028147019380854333), (893059, 0.0028117570984763592), (1714112, 0.0028003205649673245), (1715289, 0.0027908894780715203), (1714061, 0.0027882667632763997), (1715802, 0.0027882667632763997)]
<class 'tuple'>


In [18]:
top_n_authority_for_nx(a, HITS, 100)

Top 1 {'id': 1294317, 'original_url': 'https://www.flickr.com/photos/worldbank'}
weight: 0.0028239679462683694
Top 2 {'id': 612776, 'original_url': 'https://olc.worldbank.org'}
weight: 0.00282080268853487
Top 3 {'id': 165648, 'original_url': 'https://clientconnection.worldbank.org/servlet/main?menuPK=234032&pagePK=232663&piPK=234033'}
weight: 0.0028165672876734734
Top 4 {'id': 612777, 'original_url': 'https://olc.worldbank.org/'}
weight: 0.0028150531192043036
Top 5 {'id': 654094, 'original_url': 'https://policies.worldbank.org/sites/PPF3/Pages/PPFHome.aspx'}
weight: 0.0028147019380854333
Top 6 {'id': 893059, 'original_url': 'https://treasury.worldbank.org/'}
weight: 0.0028117570984763592
Top 7 {'id': 1714112, 'original_url': 'https://www.worldbank.org/en/about/careers'}
weight: 0.0028003205649673245
Top 8 {'id': 1715289, 'original_url': 'https://www.worldbank.org/en/projects-operations/products-and-services/procurement-projects-programs'}
weight: 0.0027908894780715203
Top 9 {'id': 1714

In [19]:
top_n_hub_for_nx(h, HITS, 100)

Top 1 {'id': 1714141, 'original_url': 'https://www.worldbank.org/en/about/leadership/directors/eds13'}
weight: 0.00246789907348453
Top 2 {'id': 1714148, 'original_url': 'https://www.worldbank.org/en/about/leadership/directors/eds18'}
weight: 0.00243185748867056
Top 3 {'id': 1715353, 'original_url': 'https://www.worldbank.org/en/region/sar'}
weight: 0.0024273081097951933
Top 4 {'id': 1041746, 'original_url': 'https://worldbank.org/en/newsletter-subscription'}
weight: 0.0024187786430286693
Top 5 {'id': 1715216, 'original_url': 'https://www.worldbank.org/en/newsletter-subscription?ecaturk=true?subscribe=Submit&ecaturk=true&lang=en'}
weight: 0.0024187786430286693
Top 6 {'id': 1714130, 'original_url': 'https://www.worldbank.org/en/about/leadership/directors/eds03'}
weight: 0.0024118942123601407
Top 7 {'id': 1714346, 'original_url': 'https://www.worldbank.org/en/country/chad'}
weight: 0.002411706765356004
Top 8 {'id': 1713910, 'original_url': 'https://www.worldbank.org//en/country/greece'}
w

# PageRank

In [20]:
pagerank = nx.pagerank(HITS, alpha= 0.9)

In [21]:
top_n_pagerank_for_nx(pagerank, HITS, 100)

Top 1 {'id': 1179119, 'original_url': 'https://www.cloudflare.com/5xx-error-landing'}
weight: 0.0005296341444135171
Top 2 {'id': 1451675, 'original_url': 'https://www.mediawiki.org/'}
weight: 5.324508398427455e-05
Top 3 {'id': 1034198, 'original_url': 'https://wikimediafoundation.org/'}
weight: 5.128579128645007e-05
Top 4 {'id': 1451936, 'original_url': 'https://www.mediawiki.org/wiki/Special:MyLanguage/How_to_contribute'}
weight: 5.0205662752746066e-05
Top 5 {'id': 345691, 'original_url': 'https://foundation.wikimedia.org/wiki/Cookie_statement'}
weight: 4.861798840187457e-05
Top 6 {'id': 345730, 'original_url': 'https://foundation.wikimedia.org/wiki/Privacy_policy'}
weight: 3.970889188710381e-05
Top 7 {'id': 1179121, 'original_url': 'https://www.cloudflare.com/5xx-error-landing/'}
weight: 3.4737468310655124e-05
Top 8 {'id': 1002132, 'original_url': 'https://validator.w3.org/check/referer'}
weight: 2.3122544825093982e-05
Top 9 {'id': 971828, 'original_url': 'https://twitter.com/privacy